In [4]:
# https://towardsdatascience.com/simplifing-image-outlier-detection-with-alibi-detect-6aea686bf7ba

# GPU https://www.youtube.com/watch?v=NrJz3ACosJA&t=6s conda activate py310

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import glob
import os
import shutil
from collections import Counter
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, UpSampling2D,\
    Dense, Layer, Reshape, InputLayer, Flatten, Input, MaxPooling2D
from alibi_detect.od import OutlierAE
from alibi_detect.utils.visualize import plot_instance_score, plot_feature_outlier_image

from sklearn.metrics import accuracy_score

In [3]:
# Prepare image data 
def img_to_np(path, resize = True, extract_labels=False):  
    img_array = []
    labels = []
    fpaths = glob.glob(path, recursive=True)
    for fname in fpaths:
        if(extract_labels): 
            if '_bad' in os.path.basename(fname):
                labels.append(1)  # 1 for outlier
            else:
                labels.append(0)  # 0 for non-outlier
        img = Image.open(fname).convert("L") # Grayscale when using "RGB" you have to change the encoder and decoder 
        if(resize): img = img.resize((64,64))
        img_array.append(np.asarray(img))
    images = np.array(img_array)
    if(extract_labels): return images, np.array(labels)
    return images

path_train = r'C:\Users\Ossi\Desktop\ImageClassification\elpv-dataset\train_without_bad_images\**\*'
path_test = r'C:\Users\Ossi\Desktop\ImageClassification\elpv-dataset\test_images\**\*'
path_treshold = r'C:\Users\Ossi\Desktop\ImageClassification\elpv-dataset\train_without_good_images\**\*'

train = img_to_np(path_train)
test, test_labels = img_to_np(path_test, extract_labels=True)
set_bad_threshold = img_to_np(path_treshold)
train = train.astype('float32') / 255.0
test = test.astype('float32') / 255.0
set_bad_threshold  = set_bad_threshold.astype('float32') / 255.0
# Reshape to include the channel dimension -> needed with grayscale conversion
train = np.expand_dims(train, axis=-1)
test = np.expand_dims(test, axis=-1)
set_bad_threshold = np.expand_dims(set_bad_threshold, axis=-1)

In [17]:
# Model parameters
encoding_dim = 1024
dense_dim = [8, 8, 128]

# Define the encoder
encoder_net = tf.keras.Sequential([
    Input(shape=(64, 64, 1)),  # Updated input shape for grayscale images
    Conv2D(64, 4, strides=2, padding='same', activation='relu'),
    Conv2D(128, 4, strides=2, padding='same', activation='relu'),
    Conv2D(512, 4, strides=2, padding='same', activation='relu'),
    Flatten(),
    Dense(encoding_dim)
])

# Define the decoder
decoder_net = tf.keras.Sequential([
    Input(shape=(encoding_dim,)),
    Dense(np.prod(dense_dim)),
    Reshape(target_shape=dense_dim),
    Conv2DTranspose(256, 4, strides=2, padding='same', activation='relu'),
    Conv2DTranspose(64, 4, strides=2, padding='same', activation='relu'),
    Conv2DTranspose(1, 4, strides=2, padding='same', activation='sigmoid')  # Updated output channels to 1
])

# Create the OutlierAE model
od = OutlierAE(
    threshold=0.001,
    encoder_net=encoder_net,
    decoder_net=decoder_net
)

# Compile and train the model
adam = tf.keras.optimizers.Adam(learning_rate=1e-4)
od.fit(train, epochs=10, verbose=True, optimizer=adam)

15/15 [=] - 0s 29ms/step - loss_ma: 0.0100


In [23]:
# Infer threshold on test set
od.infer_threshold(set_bad_threshold, threshold_perc=95)

# Test the model on the test set
preds = od.predict(test, outlier_type='instance',
                   return_instance_score=True,
                   return_feature_score=True)

predicted_labels = preds['data']['is_outlier']

# Calculate accuracy
accuracy = accuracy_score(test_labels, predicted_labels)
print(f'Accuracy: {accuracy * 100:.2f}%')


Accuracy: 52.98%
